Re: discussion w/ Hannah
1. Parse input: a bunch of tag names. No context. Assume that there is a structured naming convention
2. Find recurring patterns to match names and delimiters from convention
3. Find unique identifiers? Do we need them later? That'd be the moment to do it, maybe.
4. Search for abbreviations that match BRICK types
5. Return a list of matches and no matches
6. Revise and go to step 1

In [1]:
import xml.etree.ElementTree as ET
import os

In [7]:
xml_in = os.path.abspath(os.path.join(os.path.dirname(os.path.curdir), '..', 'dataset', 'CERL_027_AFXML.xml'))
tree_in = ET.parse(xml_in)
tags = []

for elem in tree_in.iter():
    if str(elem.tag) == 'Name':
        tags.append(elem.text)

In [13]:
print(tags)
print(len(tags))

['Bldg-O27', 'Is fed by', 'Meter', 'Floor_1', 'CRAC-1', 'CRAC-1_FAN-STAT', 'CRAC-1_ZN-H', 'CRAC-1_ZN-H-STPT', 'CRAC-1_ZN-T', 'Zone_High_Humidity_Flag', 'Zone_High_Temperature_Flag', 'Zone_High_Humidity', 'Zone_High_Temperature', 'CRAC-2', 'CRAC-2_FAN-STAT', 'CRAC-2_ZN-H', 'CRAC-2_ZN-H-STPT', 'CRAC-2_ZN-T', 'CRAC-3', 'CRAC-3_FAN-STAT', 'CRAC-3_ZN-H', 'CRAC-3_ZN-H-STPT', 'CRAC-3_ZN-T', 'CRAC-4', 'CRAC-4_FAN-STAT', 'CRAC-4_ZN-H', 'CRAC-4_ZN-H-STPT', 'CRAC-4_ZN-T', 'CRAC-5', 'CRAC-5_FAN-STAT', 'CRAC-5_ZN-H', 'CRAC-5_ZN-H-STPT', 'CRAC-5_ZN-T', 'CRAC-7', 'CRAC-7_FAN-STAT', 'CRAC-7_ZN-H', 'CRAC-7_ZN-H-STPT', 'CRAC-7_ZN-T', 'CRAC-8', 'CRAC-8_FAN-STAT', 'CRAC-8_ZN-H', 'CRAC-8_ZN-H-STPT', 'CRAC-8_ZN-T', 'CRAC-9', 'CRAC-9_FAN-STAT', 'CRAC-9_ZN-H', 'CRAC-9_ZN-H-STPT', 'CRAC-9_ZN-T', 'FCU-1', 'FCU-1_CHWV-FDBK', 'FCU-1_CHWV-SIG', 'FCU-1_FAN', 'FCU-1_FAN-STAT', 'FCU-1_HWV-FDBK', 'FCU-1_HWV-SIG', 'FCU-1_OCC-STAT', 'FCU-1_RunHours', 'FCU-1_ZN-CO2', 'FCU-1_ZN-CO2-STPT', 'FCU-1_ZN-H', 'FCU-1_ZN-H-STPT', 

In [10]:
def intersect_strs(s1, s2, length):
    # Find the intersection between strings s1 and s2 with a minimal length of 'length' 
    # This function is dangerous, and the 'length' parameter only gives you the feeling you are in control.

    combinations = []
    possibilities = []
    # We generate all possible combinations of n consecutive letters of the first string s1, with n € [length, len(s1)]
    for i in reversed(range(length, len(s1))):
        combinations.extend([s1[a:a+i+1] for a in range(len(s1)-i)])

    # We check if one of these combinations belongs to the second string, s2
    for c in combinations:
        if c in s2:
            possibilities.append(c)
    res = []

    # In a futile attempt to keep control over false positives, such as the detection of common patterns 
    # (e.g., the, and, tha, ent, ing, ion, tio, for, nde, has, nce, edt, tis, oft, sth, men, which are all trigrams, which, yes, is a cool new word I didn't know, and also the last trigram in this list of most common trigrams is 'men' which would be a good occasion to talk about our society, but this is a notebook on finding naming conventions in an arbitrary lists of strings.)
    # we try to keep track of the patterns we have alredy seen at least once. 
    for p in possibilities:
        notp = [i for i in possibilities if i != p]
        p_in_notp = [p in np for np in notp]
        if not(any(p_in_notp)):
            res.append(p)
    return res

def brute_force_this_please(tags, min_length=1):
    # As the name implies, we just try to figure out if a common substring exists between two strings, then if this substring
    # also exists in other strings, we have a winner. In theory. TODO: Figure out why it doesn't work.

    all_patterns = []
    for i, tag in enumerate(tags):
        print(f'\rProgress: {round(100 * i/len(tags), 2)}%', end='')
        for othertag in [j for j in tags if j != tag]:
            intersect = intersect_strs(tag, othertag, length=min_length-1)
            if intersect != []:
                all_patterns.extend([its for its in intersect if its not in all_patterns])
    return all_patterns



In [15]:
print('Finding patterns\n')
patterns = brute_force_this_please(tags + ['Thank you!'], min_length=3)


Finding patterns

Progress: 99.97%

In [16]:
print(len(patterns))

3588


In [19]:
import re


In [20]:
# And now we remove those odd 'patterns' with just numbers and symbols. How do we catch those generic patterns in a generic way?

an_patterns = [p for p in patterns if re.search('[a-zA-Z]', p)]
print(len(an_patterns))


3481


In [2]:
from rdflib import RDFS, RDF, Namespace, Graph, URIRef
brick = Graph()
brick.parse('reference/Brick.ttl')
btypes = []
for s in brick.subjects():
    try:
        btypes.append(s.split('#')[1])
    except:
        pass

    
def is_brick(a):
    res = re.process.extract(a, btypes)
    if len(res) > 0:
        print(f"Best matches for {a}: {res[:5]}")
    else:
        print(f"No match for {a}")
        return None

In [4]:
for p in an_patterns:
    is_brick(p)

NameError: name 'patterns' is not defined